In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
import warnings
warnings.filterwarnings("ignore")


In [53]:
data= pd.read_excel(r'C:\Users\Usuario\OneDrive\Desktop\Claro Insurance\Data Science Projects\04.11.24 Support Vector Machine\data releases 04.15.24.xlsx',sheet_name='Train')
predict= pd.read_excel(r'C:\Users\Usuario\OneDrive\Desktop\Claro Insurance\Data Science Projects\04.11.24 Support Vector Machine\data releases 04.15.24.xlsx',sheet_name='Predict')

In [33]:
df_filtrado = data[data['Reason for Request'].isin(['Specific request Agency', 'Release Requested/ Whit out reason', 'Comfort Release',
              'Another FMO', 'Independet FMO', 'Internal movement', 'Needed Accompaiment'])]

In [35]:
lemma = WordNetLemmatizer()
swords = stopwords.words("english")
cleanedData=[]

for text in df_filtrado["Observaciones.English"]:
    
    text = re.sub(r'http\S+', '', text) 
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]
    text = [word for word in text if word not in swords]
    text = " ".join(text)
    cleanedData.append(text)
    
df_filtrado["Cleaned Data.English"] = cleanedData

In [ ]:
# Definir los hiperparámetros que deseas ajustar y sus rangos
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'gamma': [0.1, 0.01, 0.001],  # Parámetro del kernel (solo para kernels 'rbf', 'poly', 'sigmoid')
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # Tipo de kernel
}

# Inicializar el GridSearchCV con el modelo SVM y el espacio de hiperparámetros
grid_search = GridSearchCV(SVC(class_weight=class_weights, probability=True), param_grid, cv=5, n_jobs=-1)

# Ajustar el GridSearchCV a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)

# Utilizar el modelo con los mejores hiperparámetros para hacer predicciones en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluar el rendimiento del modelo con los mejores hiperparámetros
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


In [36]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_filtrado['Cleaned Data.English'])
y = df_filtrado['Reason for Request']

# División de datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo SVM
class_weights = 'balanced'

model = SVC(kernel='linear', class_weight=class_weights,probability=True)
model.fit(X_train, y_train)

# Predicción en conjunto de prueba
y_pred = model.predict(X_test)

# Evaluación del modelo
print(classification_report(y_test, y_pred))


                                    precision    recall  f1-score   support

                       Another FMO       0.50      0.33      0.40         3
                   Comfort Release       1.00      1.00      1.00         1
                    Independet FMO       1.00      0.50      0.67         2
                 Internal movement       0.50      1.00      0.67         1
               Needed Accompaiment       1.00      1.00      1.00         2
Release Requested/ Whit out reason       1.00      0.50      0.67         2
           Specific request Agency       0.67      0.86      0.75         7

                          accuracy                           0.72        18
                         macro avg       0.81      0.74      0.74        18
                      weighted avg       0.76      0.72      0.71        18

Accuracy of model is 72.22222222222221%


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

print("Accuracy of model is {}%".format(accuracy_score(y_test,y_pred ) * 100))

In [37]:
from sklearn.metrics import roc_auc_score

# Obtener las probabilidades de predicción para las clases
y_probs = model.predict_proba(X_test)

# Calcular el AUC
auc = roc_auc_score(y_test, y_probs, average='weighted', multi_class='ovr')

print("AUC score:", auc)


AUC score: 0.6313552188552188


In [60]:
predict['Observaciones.English'] = predict['Observaciones.English'].astype(str)

lemma = WordNetLemmatizer()
swords = stopwords.words("english")
cleanedData=[]

for text in predict['Observaciones.English']:
    
    text = re.sub(r'http\S+', '', text)
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]
    text = [word for word in text if word not in swords]
    text = " ".join(text)
    cleanedData.append(text)

predict["Cleaned Data.English"] = cleanedData
X_nuevo = vectorizer.transform(predict['Cleaned Data.English'])
y_pred_nuevo = model.predict(X_nuevo)
predict['Clasificacion'] = y_pred_nuevo


predict.to_excel('resultados modelo 4.15.24.xlsx')

           NAME Create_Date         NPN                   NAME AGENT  \
0      201909-0  2019-07-08  18597037.0                 Miriam Nunez   
1      201909-1  2019-07-25  18883037.0  Maria Cristina Boscan Devis   
2     201909-10  2019-07-25  11058377.0              Yolanda Navarro   
3     201909-11  2019-07-25  18193311.0               Darline Candio   
4     201909-12  2019-09-19  15791397.0             Armando Portillo   
..          ...         ...         ...                          ...   
117  202210-141  2022-10-07  20124275.0         Yadira Perez Mendoza   
118  202210-142  2022-10-12  19074059.0       Amalia Alvarez Tricera   
119  202211-143  2022-11-10    626257.0           Lorri Ann Zelaznik   
120  202212-144  2022-12-12  17355647.0            Sabrina Contreras   
121   202005-39  2020-05-29         NaN                          NaN   

                         ACCOUNT_NAME TIPO AGENTE  Reason for Request  \
0                     Claro Insurance     Directo             